In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import numpy as np

# Experiment Time

In [4]:
from module.layer import Dense
from module.losses import meansquarederror,softmaxcrossentropy
from module.optimizers import optimizer,SGD,SGDMomentum
from module.activation import linear,tanh,sigmoid
from module.network import NeuralNetwork
from module.trainer import Trainer
from module.dropout import DropOut
from module.utils import softmax

In [5]:
from module import mnist

# lets do some data processing and cleaning

In [6]:
#from keras.datasets import mnist
#(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [7]:
X_train, y_train, X_test, y_test = mnist.load()

In [8]:
len(X_train),len(X_test)

(60000, 10000)

In [9]:
X_train.shape,X_test.shape

((60000, 784), (10000, 784))

In [10]:
y_train.shape,y_test.shape

((60000,), (10000,))

In [11]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

In [12]:
y_train.shape,y_test.shape

((60000, 1), (10000, 1))

In [15]:
#one hot encoding of the output

#for training
train_count=len(X_train)
train_labels=np.zeros((train_count,10))
for i in range(train_count):
    train_labels[i][y_train[i]]=1
#for testing
test_count=len(X_test)
test_labels=np.zeros((test_count,10))
for i in range(test_count):
    test_labels[i][y_test[i]]=1

In [16]:
train_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [17]:
train_labels.shape,test_labels.shape

((60000, 10), (10000, 10))

In [18]:
test_labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
test_labels.shape

(10000, 10)

# MNIST DEMO

In [20]:
X_train,X_test=X_train-np.mean(X_train),X_test-np.mean(X_train)
X_train,X_test=X_train/np.std(X_train),X_test/np.std(X_train)

In [21]:
np.min(X_train)

-0.424073894391566

In [22]:
y_test.shape

(10000, 1)

In [23]:
np.max(X_train)

2.821543345689335

In [24]:
#def calc_accuracy_model(model,x_test,y_test):
#    return print(f'''The model validation accuracy is: {np.equal(np.argmax(model.forward(x_test),axis=1), y_test).sum()*100.0/ x_test.shape[0]:.2f}%''')

In [25]:

def calc_accuracy_model(model, test_set, test_labels):
    model_output = model.forward(test_set)
    
    # Convert one-hot encoded labels to integer labels
    predicted_labels = np.argmax(model_output, axis=1)
    true_labels = np.argmax(test_labels, axis=1)

    correct_predictions = np.equal(predicted_labels, true_labels)
    
    accuracy = correct_predictions.sum() * 100.0 / test_set.shape[0]
    return print(f'''The model validation accuracy is: {accuracy:.2f}%''')


# Using different types of Activation function

## using Sigmoid Activation Function 

In [26]:
model=NeuralNetwork(
    layers=[
        Dense(neurons=89,activation=tanh()),
        Dense(neurons=10,activation=sigmoid())
    ],
    loss=meansquarederror(normalize=False)
)

trainer=Trainer(model,SGD(0.1))
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 0.991
validation loss after 20 is 0.975
validation loss after 30 is 0.966
validation loss after 40 is 0.954
validation loss after 50 is 0.939
training complete



In [27]:
calc_accuracy_model(model,X_test,y_test)

The model validation accuracy is: 21.57%


## lets not normalize the output

In [28]:

model=NeuralNetwork(
    layers=[
        Dense(neurons=89,activation=tanh()),
        Dense(neurons=10,activation=sigmoid())
    ],
    loss=meansquarederror()
)

trainer=Trainer(model,SGD(0.1))
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 0.991
validation loss after 20 is 0.975
validation loss after 30 is 0.966
validation loss after 40 is 0.954
validation loss after 50 is 0.939
training complete



In [29]:
calc_accuracy_model(model,X_test,test_labels)

The model validation accuracy is: 13.32%


### lets use softmax cross entropy

In [30]:
model3=NeuralNetwork(
    layers=[
        Dense(neurons=89,activation=tanh()),
        Dense(neurons=10)
    ],
    loss=softmaxcrossentropy()
)

trainer=Trainer(model,SGD(0.1))
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 0.927
validation loss after 20 is 0.916
validation loss after 30 is 0.907
validation loss after 40 is 0.897
validation loss after 50 is 0.886
training complete



In [31]:
calc_accuracy_model(model3,X_test,test_labels)

The model validation accuracy is: 10.06%


# momentum

In [32]:
model4=NeuralNetwork(
    layers=[
        Dense(neurons=89,activation=tanh()),
        Dense(neurons=10)
    ],
    loss=softmaxcrossentropy()
)

trainer=Trainer(model,SGDMomentum(lr=0.1,momentum=0.9))
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 1.000
validation loss after 20 is 1.000

loss increased after 30,final loss was 1.000, 
 we are using the model from epoch 20
training complete



In [33]:
calc_accuracy_model(model4,X_test,test_labels)

The model validation accuracy is: 10.06%


# with learning rate decay

## with linear decay type

In [34]:
model4=NeuralNetwork(
    layers=[
        Dense(neurons=89,activation=tanh()),
        Dense(neurons=10)
    ],
    loss=softmaxcrossentropy()
)
optimizer=SGDMomentum(lr=0.15,final_lr=0.01,momentum=0.9,decay_type="linear")
trainer=Trainer(model,optimizer)
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 1.000


C:\Users\nebiy\Documents\deep_learning_scratch\module\activation.py:19: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-1.0*self.input_))



loss increased after 20,final loss was 1.000, 
 we are using the model from epoch 10
training complete



In [35]:
calc_accuracy_model(model4,X_test,test_labels)

The model validation accuracy is: 10.06%


## with Exponential decay type

In [36]:
model5=NeuralNetwork(
    layers=[
        Dense(neurons=89,activation=tanh()),
        Dense(neurons=10)
    ],
    loss=softmaxcrossentropy()
)
optimizer=SGDMomentum(lr=0.15,final_lr=0.01,momentum=0.9,decay_type="Exponential")
trainer=Trainer(model,optimizer)
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 1.000

loss increased after 20,final loss was 1.000, 
 we are using the model from epoch 10
training complete



In [37]:
calc_accuracy_model(model5,X_test,test_labels)

The model validation accuracy is: 10.06%


# using the Glorort weight initialization

In [38]:
### with Expoential learing rate decay

In [39]:
model6=NeuralNetwork(
    layers=[
        Dense(neurons=89,activation=tanh(),weight_init="glorot"),
        Dense(neurons=10,weight_init="glorot")
    ],
    loss=softmaxcrossentropy(),
    seed=20190119
)
optimizer=SGDMomentum(lr=0.15,final_lr=0.01,momentum=0.9,decay_type="Exponential")
trainer=Trainer(model,optimizer)
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 1.000

loss increased after 20,final loss was 1.000, 
 we are using the model from epoch 10
training complete



In [40]:
calc_accuracy_model(model6,X_test,test_labels)

The model validation accuracy is: 10.31%


## with linear learning rate decay

In [41]:
model7=NeuralNetwork(
    layers=[
        Dense(neurons=89,activation=tanh(),weight_init="glorot"),
        Dense(neurons=10,weight_init="glorot")
    ],
    loss=softmaxcrossentropy(),
    seed=20190119
)
optimizer=SGDMomentum(lr=0.15,final_lr=0.01,momentum=0.9,decay_type="linear")
trainer=Trainer(model,optimizer)
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 1.000

loss increased after 20,final loss was 1.000, 
 we are using the model from epoch 10
training complete



In [42]:
calc_accuracy_model(model7,X_test,test_labels)

The model validation accuracy is: 10.31%


# lets see what happens when adding a dropout layer

In [43]:
model8=NeuralNetwork(
    layers=[
        Dense(neurons=178,
              activation=tanh(),
              weight_init="glorot",
             dropout=0.8),
        Dense(neurons=178,
              activation=tanh(),
              weight_init="glorot",
             dropout=0.8),
        Dense(neurons=10,
              activation=linear(),
              weight_init="glorot",
             dropout=0.8),
        
    ],
    loss=softmaxcrossentropy(),
    seed=20190119
)
optimizer=SGDMomentum(lr=0.15,final_lr=0.01,momentum=0.9,decay_type="linear")
trainer=Trainer(model8,optimizer)
trainer.fit(X_train,
    train_labels,
    X_test,
    test_labels,epochs=50,eval_every=10,seed=42,batch_sz=60
           );
       
print()

validation loss after 10 is 372645.767

loss increased after 20,final loss was 372645.767, 
 we are using the model from epoch 10
training complete

